In [ ]:
# Jetson Camera Toolkit - 多 CSI 摄像头预览
import sys
import os

# 添加 utils 目录到路径
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath("."))))

import cv2
import numpy as np
import threading
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact

from camera import VideoSource, build_csi_pipeline
from rtsp import CameraLayout

In [ ]:
# 摄像头配置
CSI_SENSOR_LEFT = 0   # 左摄像头 ID
CSI_SENSOR_RIGHT = 1  # 右摄像头 ID

# 分辨率选项
resolution_options = {
    "640x480": (640, 480),
    "1280x720": (1280, 720),
    "1920x1080": (1920, 1080),
}

# 当前分辨率
width, height = 640, 480

In [ ]:
# UI 控件配置
button_layout = widgets.Layout(width='200px', height='50px', align_self='center')
output = widgets.Output()

# 退出按钮
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)

# 图像显示控件
imgbox = widgets.Image(format='jpg', width=width*2, height=height, layout=widgets.Layout(align_self='center'))

# 布局
controls_box = widgets.VBox([imgbox, exit_button], layout=widgets.Layout(align_self='center'))

# 全局状态
running = True
current_width, current_height = width, height

def exit_button_Callback(value):
    global running
    running = False
    with output:
        print("退出...")

exit_button.on_click(exit_button_Callback)

In [ ]:
# 分辨率选择器
def change_resolution(resolution):
    global current_width, current_height
    current_width, current_height = resolution_options[resolution]
    with output:
        print(f"分辨率已更改为: {current_width}x{current_height}")

interact(
    change_resolution,
    resolution=widgets.Dropdown(
        options=resolution_options.keys(),
        value="640x480",
        description="分辨率: "
    ),
)

In [ ]:
# 图像捕获函数
def capture_frames():
    global running, current_width, current_height
    
    # 使用 toolkit 的 VideoSource
    left_src = VideoSource(f"csi://{CSI_SENSOR_LEFT}", current_width, current_height, 30)
    right_src = VideoSource(f"csi://{CSI_SENSOR_RIGHT}", current_width, current_height, 30)
    
    if not left_src.open() or not right_src.open():
        with output:
            print("无法打开摄像头")
        return
    
    with output:
        print(f"双摄像头已打开: {current_width}x{current_height} @ 30fps")
    
    while running:
        try:
            ret1, frame1 = left_src.read()
            ret2, frame2 = right_src.read()
            
            if not ret1 or not ret2:
                break

            # 水平拼接两个图像
            combined_image = np.hstack((frame1, frame2))
            
            # 调整显示大小
            display_frame = cv2.resize(combined_image, (current_width * 2, current_height))
            imgbox.value = cv2.imencode(".jpg", display_frame)[1].tobytes()

        except KeyboardInterrupt:
            break
    
    left_src.release()
    right_src.release()
    with output:
        print("摄像头已释放")

# 显示控件
display(controls_box, output)

# 在新线程中捕获图像
threading.Thread(target=capture_frames, daemon=True).start()